# CPATF: Code-switched Parliament-Aware Token Filtering

## Definition
**CPATF** (Code-switched Parliament-Aware Token Filtering) is a domain-specific preprocessing module designed for bilingual (Malay-English) Malaysian parliamentary Hansard texts. It filters out noise (e.g., attendance lists, OCR artifacts, repeated honorifics) while selectively retaining content-rich tokens essential for downstream topic modeling, entity extraction, and speaker identification.

## Core Concept
Hansard proceedings are highly structured yet noisy due to:
- Frequent code-switching between Malay and English
- Repetitive honorific titles (YB, Dato', Datuk Seri, etc.)
- Historical OCR errors and attendance lists in older documents (1970s–1990s)

CPATF addresses these challenges by assigning each token a **retention score** based on linguistic, syntactic, and domain-specific signals, preserving only high-information tokens.

## Processing Flow
1. **Attendance List Detection**  
   Skip entire segments identified as attendance lists using regex patterns (e.g., multiple ". " separated names or numbered lines).

2. **Tokenization & Linguistic Analysis**  
   - spaCy pipeline with custom Malay POS fallback  
   - FastText language identification per token (ms/en confidence > 0.7)

3. **Retention Score Calculation**  
   Compute score for each token and retain only those exceeding threshold.

4. **Lemmatization & Output**  
   Lemmatize retained tokens and concatenate into cleaned text.

## Mathematical Formulation
For each token \( t \):

Retention score:
$$
r(t) = w_1 \cdot \mathbb{I}_{\text{lang}}(t) + w_2 \cdot \mathbb{I}_{\text{POS}}(t) + w_3 \cdot \mathbb{I}_{\text{NER}}(t) - w_4 \cdot \text{red}(t)
$$

Token is retained if \( r(t) \geq \theta \).

### Indicators:
- **I_lang(t)**: 1 if FastText predicts "ms" or "en" with confidence > 0.7, else 0
- **I_POS(t)**: 1 if POS ∈ {NOUN, PROPN, VERB, ADJ, ADV}, else 0
- **I_NER(t)**: 1 if NER type ∈ {PERSON, ORG, LOC, BILL/LAW}, else 0
- **red(t)**: Redundancy penalty = (sum of repeated honorifics beyond first occurrence in local window) / window length

### Optimized Parameters (via ablation study on core500 dataset):
- w₁ = 0.25, w₂ = 0.25, w₃ = 0.40, w₄ = 0.10
- Threshold θ = 0.6

## Role in Overall Pipeline
CPATF produces clean, content-focused segments that serve as standardized input for all topic modeling pipelines, ensuring fair evaluation and high-quality input for MEHTC clustering and XLM-RoBERTa fine-tuning.

### ===========================================================================================================================================
### Imports and Environment Setup

In [2]:
import os
import re
import gc
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Optional
from datetime import datetime
from pathlib import Path
from functools import lru_cache

import pymongo
import spacy
import fasttext
import random
from tqdm import tqdm
from dotenv import load_dotenv

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Load environment variables
project_root = Path.cwd().parents[1] if 'parents' in dir(Path.cwd()) else Path.cwd()
backend_env_path = project_root / "3_app_system" / "backend" / ".env"
load_dotenv(backend_env_path)

# FastText language model
FASTTEXT_MODEL_PATH = 'lid.176.bin'
if not os.path.exists(FASTTEXT_MODEL_PATH):
    print("Downloading FastText language model...")
    import urllib.request
    urllib.request.urlretrieve("https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin", FASTTEXT_MODEL_PATH)

# Load FastText model
ft_model = fasttext.load_model(FASTTEXT_MODEL_PATH)

# Load spaCy models
nlp_ner = spacy.load("xx_ent_wiki_sm")      # Multilingual NER
nlp_en = spacy.load("en_core_web_sm")       # English lemmatizer

# Common Malay function words (for rule-based POS filtering)
MALAY_FUNCTION_WORDS = {
    'yang', 'di', 'dan', 'untuk', 'dari', 'pada', 'dalam', 'adalah', 'ialah', 'bahawa',
    'sebagai', 'oleh', 'kepada', 'dengan', 'atau', 'jika', 'kerana', 'supaya', 'walaupun',
    'serta', 'tetapi', 'manakala', 'itu', 'ini', 'tersebut', 'akan', 'telah', 'sudah'
}

print("Environment ready.")
print(f"CPU Cores: {os.cpu_count()} ")

Environment ready.
CPU Cores: 32 


### Database Connection and Data Loading

In [3]:
client = pymongo.MongoClient(os.getenv("MONGO_URI"))
db = client["MyParliament"]

# Collections
segmented_col = db["hansard_segmented500"]
honorific_col = db["honorific_dictionary"]
cpatf_col = db["hansard_cpatf500"]

# Load honorifics
honorific_dict = honorific_col.find_one({}, {"categories": 1})
if not honorific_dict:
    raise ValueError("Honorific dictionary not found")
all_honorifics = set()
for titles in honorific_dict.get("categories", {}).values():
    all_honorifics.update([t.lower() for t in titles])

print(f"Loaded {len(all_honorifics)} unique honorifics.")

# Load FastText model
ft_model = fasttext.load_model(FASTTEXT_MODEL_PATH)

# Load documents with correct date field
all_docs = list(tqdm(
    segmented_col.find(
        {}, 
        {
            "_id": 1, 
            "segmentation_output": 1,   
            "hansardDate": 1,          
            "mesyuarat": 1,
            "parlimen": 1,
            "penggal": 1,
            "decade": 1
        }
    ),
    desc="Loading segmented documents"
))

print(f"Successfully loaded {len(all_docs)} documents.")

# Check samples with correct date field
print("\nSample documents check:")
for doc in all_docs[:5]:
    print(f"Doc ID: {doc['_id']}")
    print(f"  Date: {doc.get('hansardDate', 'Missing')}")
    print(f"  Segments: {len(doc.get('segmentation_output', []))}")
    print(f"  Parlimen: {doc.get('parlimen')}, Penggal: {doc.get('penggal')}, Mesyuarat: {doc.get('mesyuarat')}")
    print(f"  Decade: {doc.get('decade')}")
    print("---")

Loaded 20 unique honorifics.


Loading segmented documents: 500it [00:14, 35.11it/s]

Successfully loaded 500 documents.

Sample documents check:
Doc ID: 6947d3ebdaaf821ec476383b
  Date: 1959-09-11 00:00:00
  Segments: 16
  Parlimen: 1, Penggal: 1, Mesyuarat: 1
  Decade: pre1970
---
Doc ID: 6947d3ebdaaf821ec476383c
  Date: 1961-10-19 00:00:00
  Segments: 67
  Parlimen: 1, Penggal: 3, Mesyuarat: 1
  Decade: pre1970
---
Doc ID: 6947d3ebdaaf821ec476383d
  Date: 1961-04-24 00:00:00
  Segments: 56
  Parlimen: 1, Penggal: 3, Mesyuarat: 1
  Decade: pre1970
---
Doc ID: 6947d3ebdaaf821ec476383e
  Date: 1961-04-28 00:00:00
  Segments: 61
  Parlimen: 1, Penggal: 3, Mesyuarat: 1
  Decade: pre1970
---
Doc ID: 6947d3ebdaaf821ec476383f
  Date: 1963-08-15 00:00:00
  Segments: 42
  Parlimen: 1, Penggal: 5, Mesyuarat: 1
  Decade: pre1970
---


### CPATF Core Functions (Bilingual Processing)

In [8]:
W_LANG = 0.30
W_POS  = 0.30
W_RED  = 0.15
THRESHOLD = 0.40 
LANG_CONF_THRESHOLD = 0.6
REDUNDANCY_WINDOW = 15

CONTENT_POS_TAGS = {'NOUN', 'PROPN', 'VERB', 'ADJ', 'ADV', 'NUM'}

def is_attendance_list(text: str) -> bool:
    dot_pattern = re.compile(r'(\.\s+[A-Z][a-z]+){5,}')
    num_pattern = re.compile(r'^\d+\.', re.MULTILINE)
    return bool(dot_pattern.search(text) or len(num_pattern.findall(text)) > 5)

@lru_cache(maxsize=30000)
def get_lang_indicator(token: str) -> int:
    pred = ft_model.predict(token.replace('\n', ' '), k=1)
    lang, conf = pred[0][0].replace('__label__', ''), pred[1][0]
    return 1 if lang in ['ms', 'en', 'id'] and conf > LANG_CONF_THRESHOLD else 0

def get_redundancy_penalty(tokens: List[str], idx: int) -> float:
    start = max(0, idx - REDUNDANCY_WINDOW // 2)
    end = min(len(tokens), idx + REDUNDANCY_WINDOW // 2 + 1)
    window = [t.lower() for t in tokens[start:end]]
    repeated = sum(max(0, window.count(h) - 1) for h in all_honorifics if h in window)
    return min(repeated * 0.15, 0.4)  # Stronger penalty for honorifics

def simple_malay_stem(word: str) -> str:
    word_lower = word.lower()
    if len(word_lower) <= 4:
        return word_lower
    suffixes = ['kan', 'an', 'i', 'lah', 'kah', 'nya', 'tah', 'pun', 'mu', 'ku']
    for suffix in suffixes:
        if word_lower.endswith(suffix):
            return word_lower[:-len(suffix)]
    return word_lower

def rule_based_pos(word: str) -> str:
    if word and word[0].isupper():
        return 'PROPN'
    if word.isdigit() or re.match(r'^\d', word):
        return 'NUM'
    if word.lower().endswith(('kan', 'i', 'lah', 'nya', 'tah')):
        return 'VERB'
    return 'NOUN'

def process_segment(segment: str) -> str:
    if isinstance(segment, list):
        segment = " ".join([s.strip() for s in segment if s.strip()])

    if not segment or not segment.strip():
        return ""

    if is_attendance_list(segment):
        return ""

    segment = segment[:6000]
    words = segment.split()
    if not words:
        return ""

    pos_tags = [rule_based_pos(word) for word in words]

    retained = []
    for idx, word in enumerate(words):
        lang_ind = get_lang_indicator(word)
        pos_ind = 1 if pos_tags[idx] in CONTENT_POS_TAGS else 0
        red_pen = get_redundancy_penalty(words, idx)

        score = W_LANG * lang_ind + W_POS * pos_ind - W_RED * red_pen

        # Force retain only critical entities
        force_retain = (
            pos_tags[idx] == 'PROPN' or 
            pos_tags[idx] == 'NUM' or 
            len(word) > 8  # Longer words likely entities
        )

        if force_retain or score >= THRESHOLD:
            word_lower = word.lower()
            if lang_ind == 1 and not word[0].isupper():
                normalized = simple_malay_stem(word_lower)
            else:
                normalized = word_lower
            retained.append(normalized)

    return " ".join(retained)

def process_long_segment(segment: str, max_chunk_tokens: int = 2000) -> str:
    words = segment.split()
    total_tokens = len(words)
    
    if total_tokens <= max_chunk_tokens:
        return process_segment(segment)
    
    print(f"Long segment ({total_tokens} tokens) → chunking (max {max_chunk_tokens})...")
    
    retained_words = []
    overlap = 200
    start = 0
    while start < total_tokens:
        end = min(start + max_chunk_tokens, total_tokens)
        chunk_text = " ".join(words[start:end])
        cleaned_chunk = process_segment(chunk_text)
        chunk_words = cleaned_chunk.split()
        if chunk_words:
            if retained_words and chunk_words[:50] == retained_words[-50:]:
                retained_words.extend(chunk_words[50:])
            else:
                retained_words.extend(chunk_words)
        start = end - overlap if end < total_tokens else end
    
    return " ".join(retained_words)

### Test on 10 Random Real Segments

In [9]:
# Randomly sample 10 documents for testing
test_docs = random.sample(all_docs, min(10, len(all_docs)))

print("=== CPATF Test: Before & After on 10 Random Real Segments ===\n")

total_before = 0
total_after = 0
valid_count = 0
total_entities_before = 0
total_entities_after = 0

for i, doc in enumerate(test_docs, 1):
    doc_id = doc["_id"]
    date_str = doc.get("hansardDate")
    if isinstance(date_str, datetime):
        date_str = date_str.strftime("%Y-%m-%d")
    else:
        date_str = str(date_str) if date_str else "Unknown Date"
    
    raw_segments = doc.get("segmentation_output", [])

    print(f"[{i}/10] Doc ID: {doc_id} | Date: {date_str}")
    print(f"Raw segmentation_output items: {len(raw_segments)}\n")

    # Extract clean text strings
    segments = []
    for item in raw_segments:
        if isinstance(item, str):
            text = item.strip()
        elif isinstance(item, dict):
            text = item.get("text", "") or item.get("content", "")
            text = text.strip()
        else:
            continue
        if text:
            segments.append(text)

    if not segments:
        print("No valid text segments found in this document.\n")
        print("-" * 90 + "\n")
        continue

    valid_count += 1

    # Pick the longest segment for demonstration
    sample_seg = max(segments, key=len)

    print("【Original Segment】")
    print(sample_seg[:600] + ("..." if len(sample_seg) > 600 else "") + "\n")

    # Use robust chunking - rule-based is fast, so larger chunks OK
    cleaned = process_long_segment(sample_seg, max_chunk_tokens=1000)

    print("【CPATF Cleaned (Rule-based)】")
    print(cleaned[:600] + ("..." if len(cleaned) > 600 else "") + "\n")

    # Token count
    before_tokens = len(sample_seg.split())
    after_tokens = len(cleaned.split())
    total_before += before_tokens
    total_after += after_tokens

    # Simple entity preservation check (using spaCy NER on original and cleaned)
    try:
        orig_doc = nlp_ner(sample_seg)
        clean_doc = nlp_ner(cleaned)
        
        orig_entities = {ent.text.lower() for ent in orig_doc.ents 
                        if ent.label_ in ['PERSON', 'ORG', 'LOC']}
        clean_entities = {ent.text.lower() for ent in clean_doc.ents 
                         if ent.label_ in ['PERSON', 'ORG', 'LOC']}
        
        preserved = len(orig_entities & clean_entities)
        total_entities_before += len(orig_entities)
        total_entities_after += preserved
        
        print(f"Entities preserved: {preserved}/{len(orig_entities)} "
              f"({100 * preserved / len(orig_entities):.1f}% if orig_entities else 0) ")
        if orig_entities:
            print(f"Sample entities kept: {list(orig_entities & clean_entities)[:5]}")
    except:
        print("Entity check skipped (NER error)")

    reduction = before_tokens - after_tokens
    reduction_pct = (reduction / before_tokens * 100) if before_tokens > 0 else 0
    print(f"Tokens: {before_tokens} → {after_tokens} "
          f"(reduced by {reduction}, {reduction_pct:.1f}%)")
    print("-" * 90 + "\n")

# Final Summary
print("=== CPATF Test Summary ===")
print(f"Documents with valid text: {valid_count}/10")

if total_before > 0:
    overall_reduction = total_before - total_after
    overall_pct = 100 * overall_reduction / total_before
    entity_preserve_rate = (100 * total_entities_after / total_entities_before 
                           if total_entities_before > 0 else 0)
    
    print(f"Total tokens before CPATF : {total_before}")
    print(f"Total tokens after CPATF  : {total_after}")
    print(f"Overall token reduction   : {overall_reduction} tokens ({overall_pct:.1f}%)")
    print(f"Entity preservation rate  : {entity_preserve_rate:.1f}% "
          f"({total_entities_after}/{total_entities_before} entities kept)")
    print(f"\n→ Reduction ~{overall_pct:.1f}% with ~{entity_preserve_rate:.1f}% entity retention")
else:
    print("No valid tokens found in sampled documents.")

print("\n=== CPATF Test Completed Successfully ===")

=== CPATF Test: Before & After on 10 Random Real Segments ===

[1/10] Doc ID: 6947dae7daaf821ec47638bb | Date: 1981-10-27
Raw segmentation_output items: 93

【Original Segment】
Terima kasih, hend Tuan Yang di-Pertua. Oleh yang demikian iala bagi menyelesaikan Rancangan Makanan seko Tambahan yang sangat berfaedah ini adalah muri dicadangkan mana-mana sekolah yang Wala terpilih hendaklah diberi kepada semua murid dala sekolah tersebut dan yang kedua, bantuan 25 ingi sen itu hendaklah ditimbangkan semula oleh S kerana memandangkan harga bahan-bahan kera makanan yang melambung hari ini adalah seba tidak sesuai. mera Tuan Yang di-Pertua, berhubung dengan pent Skim Pinjaman Buku Teks pula, saya hendak seba menyentuh dan memberi sedikit pandangan akan iaitu mengikut pengala...

Long segment (5716 tokens) → chunking (max 1000)...
【CPATF Cleaned (Rule-based)】
terima kasih, tuan yang di-pertua. oleh yang iala bagi menyelesaikan rancangan makanan tambahan yang sangat berfaedah ini ada dicadangkan 

Entities preserved: 27/120 (22.5% if orig_entities else 0) 
Sample entities kept: ['mekah', 'libya', 'keselamatan', 'umpamanya', 'akan']
Tokens: 5716 → 3935 (reduced by 1781, 31.2%)
------------------------------------------------------------------------------------------

[2/10] Doc ID: 6947dae7daaf821ec47638cb | Date: 1982-03-19
Raw segmentation_output items: 174

【Original Segment】
apa guna undi kepada Kerajaan, apa dia boleh bagi? Kalau undi kami pun sama juga. Dalam keadaan semacam itu, dia seolah-olah menggalak pengundi-pengundi bertanya 1339 19 MAC kepada calon-calon Kerajaan, apa Kerajaan m hendak bagi. Bila pengundi-pengundi o bertanya maka terpaksa Kerajaan membuat s penerangan. Calon Kerajaan ini bertanya. r kalau kami jadi Kerajaan maka kami dapat t mengadakan ini, ini dan ini, dan oleh kerana s pengundi mendapat ini dianya berpendapat m bahawa satu cara untuk mengugut calon- p calon Kerajaan ialah dengan berkata, “Kalau tidak bagi ini, kamu undi parti Pembangka...

Long se

### Full Preprocessing on 500 Documents using Best CPATF Parameters (Multi-threaded)

In [11]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import gc
from pymongo.errors import ConnectionFailure, BulkWriteError
from pymongo import UpdateOne

# Full preprocessing - resume-safe (avoid duplicates if interrupted)

all_docs = list(segmented_col.find({}))
total_docs = len(all_docs)

if total_docs == 0:
    print("Error: No documents found in segmented_col.")
else:
    print(f"Found {total_docs} documents in segmented_col.")
    print("Resume-safe preprocessing: skipping already processed parent_doc_id")

start_time = time.time()

# Get already processed parent_doc_ids to skip
existing_parent_ids = set(
    seg["parent_doc_id"] for seg in cpatf_col.find({}, {"parent_doc_id": 1})
)
print(f"Found {len(existing_parent_ids)} already processed parent_doc_ids - skipping them")

# Filter only unprocessed docs
unprocessed_docs = [doc for doc in all_docs if doc["_id"] not in existing_parent_ids]
print(f"Need to process {len(unprocessed_docs)} new documents")

all_cleaned_segments = []
total_original_tokens = 0
total_cleaned_tokens = 0

BATCH_SIZE = 500  # Safe batch

def process_full_document_wrapper(doc):
    segments = process_full_document(doc)
    return segments

# Multi-thread processing only on unprocessed docs
if unprocessed_docs:
    with ThreadPoolExecutor(max_workers=24) as executor:
        futures = [executor.submit(process_full_document_wrapper, doc) for doc in unprocessed_docs]
        
        for future in tqdm(as_completed(futures), total=len(unprocessed_docs), desc="Processing new documents", unit="doc"):
            segments = future.result()
            all_cleaned_segments.extend(segments)
            
            for seg in segments:
                total_original_tokens += seg["original_token_count"]
                total_cleaned_tokens += seg["cleaned_token_count"]
            
            if len(all_cleaned_segments) % 2000 == 0:
                gc.collect()
else:
    print("All documents already processed - nothing to do!")

# Batch insert with upsert (avoid duplicates even if interrupted)
elapsed_time = time.time() - start_time

if all_cleaned_segments:
    print(f"\nStarting safe batch insert of {len(all_cleaned_segments)} new segments (batch size: {BATCH_SIZE})...")
    
    # Use upsert with unique key (parent_doc_id + original_text hash)
    # Create index first 
    cpatf_col.create_index([("parent_doc_id", 1), ("original_text_hash", 1)], unique=True)
    
    for i in range(0, len(all_cleaned_segments), BATCH_SIZE):
        batch = all_cleaned_segments[i:i + BATCH_SIZE]
        operations = []
        for seg in batch:
            # Add hash for dedup
            import hashlib
            text_hash = hashlib.md5(seg["original_text"].encode('utf-8')).hexdigest()
            seg["original_text_hash"] = text_hash
            
            operations.append(
                UpdateOne(
                    {"parent_doc_id": seg["parent_doc_id"], "original_text_hash": text_hash},
                    {"$setOnInsert": seg},
                    upsert=True
                )
            )
        
        retry_count = 0
        max_retries = 5
        while retry_count < max_retries:
            try:
                result = cpatf_col.bulk_write(operations, ordered=False)
                print(f"Batch {i//BATCH_SIZE + 1}: Inserted {result.upserted_count}, Modified {result.modified_count}")
                break
            except (ConnectionFailure, BulkWriteError) as e:
                retry_count += 1
                print(f"Batch failed (attempt {retry_count}/{max_retries}): {e}")
                time.sleep(5 * retry_count)
        else:
            print("Batch failed after max retries")
    
    overall_reduction = 100 * (1 - total_cleaned_tokens / total_original_tokens) if total_original_tokens > 0 else 0
    
    print(f"\n=== Full Preprocessing Complete ===")
    print(f"Total time: {elapsed_time/60:.1f} minutes")
    print(f"New segments inserted: {len(all_cleaned_segments)}")
    print(f"Overall reduction: {overall_reduction:.1f}%")
    print(f"Collection: hansard_cpatf500")
else:
    print("No new segments to process")

print("\nCPATF preprocessing done")

Found 500 documents in segmented_col.
Resume-safe preprocessing: skipping already processed parent_doc_id
Found 0 already processed parent_doc_ids - skipping them
Need to process 500 new documents
Long segment (1354 tokens) → chunking (max 1200)...
Long segment (3773 tokens) → chunking (max 1200)...
Long segment (1262 tokens) → chunking (max 1200)...
Long segment (2426 tokens) → chunking (max 1200)...
Long segment (6704 tokens) → chunking (max 1200)...
Long segment (1323 tokens) → chunking (max 1200)...
Long segment (1207 tokens) → chunking (max 1200)...
Long segment (1276 tokens) → chunking (max 1200)...
Long segment (3021 tokens) → chunking (max 1200)...
Long segment (1785 tokens) → chunking (max 1200)...
Long segment (1504 tokens) → chunking (max 1200)...
Long segment (1560 tokens) → chunking (max 1200)...
Long segment (1772 tokens) → chunking (max 1200)...
Long segment (1218 tokens) → chunking (max 1200)...
Long segment (1404 tokens) → chunking (max 1200)...
Long segment (1496 toke

Processing new documents:   4%|▎         | 18/500 [00:00<00:14, 32.83doc/s]

Long segment (1309 tokens) → chunking (max 1200)...
Long segment (1397 tokens) → chunking (max 1200)...
Long segment (1670 tokens) → chunking (max 1200)...
Long segment (1905 tokens) → chunking (max 1200)...
Long segment (2399 tokens) → chunking (max 1200)...
Long segment (1423 tokens) → chunking (max 1200)...
Long segment (1348 tokens) → chunking (max 1200)...
Long segment (2080 tokens) → chunking (max 1200)...
Long segment (1759 tokens) → chunking (max 1200)...
Long segment (1918 tokens) → chunking (max 1200)...
Long segment (1653 tokens) → chunking (max 1200)...
Long segment (1414 tokens) → chunking (max 1200)...
Long segment (1263 tokens) → chunking (max 1200)...
Long segment (8538 tokens) → chunking (max 1200)...
Long segment (1438 tokens) → chunking (max 1200)...
Long segment (6201 tokens) → chunking (max 1200)...
Long segment (1719 tokens) → chunking (max 1200)...
Long segment (2434 tokens) → chunking (max 1200)...
Long segment (2356 tokens) → chunking (max 1200)...
Long segment

Processing new documents:   5%|▍         | 24/500 [00:02<00:56,  8.47doc/s]

Long segment (1499 tokens) → chunking (max 1200)...Long segment (1962 tokens) → chunking (max 1200)...
Long segment (1246 tokens) → chunking (max 1200)...
Long segment (1452 tokens) → chunking (max 1200)...
Long segment (1355 tokens) → chunking (max 1200)...
Long segment (3638 tokens) → chunking (max 1200)...
Long segment (1281 tokens) → chunking (max 1200)...
Long segment (1545 tokens) → chunking (max 1200)...
Long segment (2828 tokens) → chunking (max 1200)...
Long segment (14768 tokens) → chunking (max 1200)...
Long segment (1661 tokens) → chunking (max 1200)...
Long segment (1254 tokens) → chunking (max 1200)...

Long segment (1500 tokens) → chunking (max 1200)...
Long segment (3448 tokens) → chunking (max 1200)...
Long segment (2098 tokens) → chunking (max 1200)...
Long segment (2819 tokens) → chunking (max 1200)...
Long segment (2264 tokens) → chunking (max 1200)...
Long segment (1253 tokens) → chunking (max 1200)...
Long segment (2395 tokens) → chunking (max 1200)...
Long segmen

...

Processing new documents:  99%|█████████▉| 496/500 [02:19<00:00, 96.42doc/s]

Long segment (1470 tokens) → chunking (max 1200)...
Long segment (1398 tokens) → chunking (max 1200)...
Long segment (1441 tokens) → chunking (max 1200)...
Long segment (1534 tokens) → chunking (max 1200)...
Long segment (1396 tokens) → chunking (max 1200)...


Processing new documents: 100%|██████████| 500/500 [02:20<00:00,  3.57doc/s]

Long segment (1298 tokens) → chunking (max 1200)...

Starting safe batch insert of 195055 new segments (batch size: 500)...


Batch 1: Inserted 500, Modified 0
Batch 2: Inserted 498, Modified 0
Batch 3: Inserted 500, Modified 0
Batch 4: Inserted 500, Modified 0
Batch 5: Inserted 497, Modified 0
Batch 6: Inserted 499, Modified 0
Batch 7: Inserted 499, Modified 0
Batch 8: Inserted 499, Modified 0
Batch 9: Inserted 500, Modified 0
Batch 10: Inserted 498, Modified 0
Batch 11: Inserted 500, Modified 0
Batch 12: Inserted 498, Modified 0
Batch 13: Inserted 500, Modified 0
Batch 14: Inserted 499, Modified 0
Batch 15: Inserted 499, Modified 0
Batch 16: Inserted 497, Modified 0
Batch 17: Inserted 499, Modified 0
Batch 18: Inserted 500, Modified 0
Batch 19: Inserted 485, Modified 0
Batch 20: Inserted 481, Modified 0
Batch 21: Inserted 499, Modified 0
Batch 22: Inserted 498, Modified 0
Batch 23: Inserted 500, Modified 0
Batch 24: Inserted 500, Modified 0
Batch 25: Inserted 497, Modified 0
Batch 26: Inserted 499, Modified 0
Batch 27: Inserted 500, Modified 0
Batch 28: Inserted 498, Modified 0
Batch 29: Inserted 498, Modif